# 2-PandasDataframe

This notebook demonstrates how to read and process a tabular datafile with the [Pandas](https://pandas.pydata.org/) dataframe library. Pandas dataframes are limited to run on a single core and data must fit into memory.

Author: Peter W. Rose (pwrose@ucsd.edu)

In [1]:
import os
import pandas as pd
import time

If LOCAL_SCRATCH_DIR environment variable is not set, this notebook accesses the ../data directory for temporary files.

In [2]:
DATA_DIR = os.getenv("LOCAL_SCRATCH_DIR", default="../data")
filename = os.path.join(DATA_DIR, "gene_info.tsv")
file_size = f"{os.path.getsize(filename)/1E9:.1f}"
RESULTS_DIR = "results"
os.makedirs(RESULTS_DIR, exist_ok=True)

### Setup Benchmark

The ```file_format``` parameter is used for benchmarking different file formats. 
The cell below has been [parameterized](https://papermill.readthedocs.io/en/latest/usage-parameterize.html#jupyterlab-3-0) as input parameters for [papermill](https://papermill.readthedocs.io/en/latest/index.html).

In [3]:
file_format = "csv"
#file_format = "parquet"

In [4]:
start = time.time()

### Read Data

In [5]:
# read only specified columns and rows
column_names = ["GeneID", "Symbol", "Synonyms", "description", "type_of_gene", "#tax_id", "chromosome"]
filters=[[("type_of_gene", "==", "protein-coding")]]

if file_format == "csv":
    filename = os.path.join(DATA_DIR, "gene_info.tsv")
    genes = pd.read_csv(filename, usecols=column_names, dtype=str, sep="\t")
    genes.query("type_of_gene == 'protein-coding'", inplace=True)
elif file_format == "parquet":
    filename = os.path.join(DATA_DIR, "gene_info.parquet")
    genes = pd.read_parquet(filename, columns=column_names, filters=filters)
else:
    print("invalid file format")
    
print("Filename:", filename)
    
genes.rename(columns={"#tax_id": "tax_id"}, inplace=True)

Filename: /scratch/pwrose/job_22854870/gene_info.tsv


In [6]:
# print(f"Total memory: {genes.memory_usage(deep=True).sum()/1E9:.1f} GB")
# Total memory: 17.6 GB

### Process Data

In [7]:
genes.head()

,tax_id,GeneID,Symbol,Synonyms,chromosome,description,type_of_gene
6,24,77267466,dnaA,N5094_00005,-,chromosomal replication initiator protein DnaA,protein-coding
7,24,77267467,dnaN,N5094_00010,-,DNA polymerase III subunit beta,protein-coding
8,24,77267468,recF,N5094_00015,-,DNA replication/repair protein RecF,protein-coding
9,24,77267469,gyrB,N5094_00020,-,DNA topoisomerase (ATP-hydrolyzing) subunit B,protein-coding
10,24,77267470,N5094_RS00025,N5094_00025,-,HDOD domain-containing protein,protein-coding


In [8]:
groups = genes.groupby(["tax_id"]).size().reset_index(name="count")
groups = groups.sort_values("count", ascending=False)

### Display Results

#### Number of human protein-coding genes (tax_id = 9606)

In [9]:
groups.query("tax_id == '9606'")

,tax_id,count
44281,9606,20648


#### Top 5 organisms with the most protein-coding genes

In [10]:
groups.head()

,tax_id,count
34950,4565,104037
32211,3708,90975
43524,90675,82686
33600,412133,72290
44082,94328,68154


In [11]:
end = time.time()

In [12]:
df = pd.DataFrame([{"cores": 1, "time": end-start, "size": file_size, "format": file_format, "dataframe": "Pandas"}])
output_file = f"2-PandasDataframe_{file_size}_{file_format}_1.csv"
df.to_csv(os.path.join(RESULTS_DIR, output_file), index=False)

In [13]:
print(f"Pandas: {end - start:.1f} sec.")

Pandas: 66.2 sec.
